In [ ]:
import sys
import os
from glob import glob
!pip install tensorflow
sys.path.append("../")
from NEATModels import NEATDetection, nets
from NEATModels.config import NeatConfig
from NEATUtils import helpers

from NEATUtils.helpers import save_json
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

     |████████████████████████████████| 394.8 MB 13 kB/s s eta 0:00:01  |▊                               | 8.4 MB 6.8 MB/s eta 0:00:58     |▉                               | 10.6 MB 6.8 MB/s eta 0:00:57     |█                               | 13.7 MB 7.9 MB/s eta 0:00:49     |██▏                             | 26.1 MB 6.6 MB/s eta 0:00:56     |███▍                            | 41.6 MB 7.0 MB/s eta 0:00:51     |████▍                           | 54.7 MB 393 kB/s eta 0:14:25     |████▌                           | 56.0 MB 8.4 MB/s eta 0:00:41     |████▋                           | 56.4 MB 8.4 MB/s eta 0:00:41     |█████▏                          | 63.6 MB 7.5 MB/s eta 0:00:45     |███████                         | 85.4 MB 7.3 MB/s eta 0:00:43     |████████                        | 97.3 MB 6.9 MB/s eta 0:00:44     |████████▋                       | 106.8 MB 8.3 MB/s eta 0:00:35     |██████████▎                     | 126.4 MB 6.3 MB/s eta 0:00:43     |███████████▉                    | 146.2 MB

     |████████████████████████████████| 77 kB 850 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 8.0 MB/s eta 0:00:01
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=77135 sha256=7d2e6bd95b29011e2bb7b93cdf6dc7c6a236bed6a190e4d158edf2999e822ec7
  Stored in directory: /home/kapoorlab/.cache/pip/wheels/5f/fd/9e/b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=bbff4ed4afcd837d505d76563781201ed71a5a27cd5b24807a55247cd0e0ae16
  Stored in directory: /home/kapoorlab/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built wrapt termcolor
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: wheel
    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully unins

In [ ]:
TrainDataDir = '/data/u934/service_imagerie/v_kapoor/CSVforNeat/YolONEAT/CenterTrainData/'
Model_dir = '/data/u934/service_imagerie/CurieTrainingDatasets/YolONEAT/CurieDeepLearningModels/YolONEAT/'
Model_Name = 'ORYolo.h5'

In [ ]:
#Neural network parameters
#For ORNET use residual = True and for OSNET use residual = False
residual = True
#NUmber of starting convolutional filters, is doubled down with increasing depth
startfilter = 48
#CNN network start layer, mid layers and lstm layer kernel size
start_kernel = 3
lstm_kernel = 3
mid_kernel = 3
#Network depth has to be 9n + 2, n= 3 or 4 is optimal for Notum dataset
depth = 29
#Training epochs, longer the better zith proper chosen leqrning rate
epochs = 150
#Size of the gradient descent length vector, start small and use callbacs to get smaller when reaching the minima
learning_rate = 1.0E-4
#For stochastic gradient decent, the batch size used for computing the gradients
batch_size = 10
#Number of LSTM hidden layers > time sequence used for training
lstm = 16
# use softmax for single event per box, sigmoid for multi event per box
multievent = False
#X Y T H W Confidence Angle makes up the box vector
categories = 6
box_vector = 7
# Grid and anchor boxes for yolo
gridX = 1
gridY = 1
nboxes = 5
#Weightage to co-ordinate loss term
lambdacord = 5
#X Y T- T+
crop_size = [256,256,4,5]
sizeX = crop_size[0]
sizeY = crops_size[1]
sizeTminus = crop_size[2]
sizeTplus = crop_size[3]

In [ ]:
config = NeatConfig(residual = residual, startfilter = startfilter, start_kernel = start_kernel, 
                    mid_kernel = mid_kernel,lstm_kernel = lstm_kernel, categories = categories,
                    box_vector = box_vector,
                    depth = depth, lstm = lstm, learning_rate = learning_rate, batch_size = batch_size,
                    epochs = epochs, ModelName = Model_Name, sizeX = sizeX, sizeY = sizeY,
                    sizeTminus = sizeTminus, sizeTplus = sizeTplus,
                    gridX = gridX, gridY = gridY, nboxes = nboxes, lambdacord = lambdacord, multievent = multievent)

config_json = config.to_json()
show = True

model_weights = Model_dir + Model_Name

if os.path.exists(model_weights):

    model_weights = model_weights
    print('loading weights')
else:
   
    model_weights = None

Categories_Name = []
Categories_Name.append(['Normal', 0])
Categories_Name.append(['Apoptosis', 1])
Categories_Name.append(['Divisions', 2])
Categories_Name.append(['MacroKitty', 3])
Categories_Name.append(['NonMatureP1', 4])
Categories_Name.append(['MatureP1', 5])
print(config)
save_json(config_json, Model_dir + Model_Name + 'Param.json')

In [ ]:
Train = NEATDetection(config, TrainDataDir, Categories_Name, box_vector, Model_dir, Model_Name, model_weights = model_weights, show = show)

Train.loadData()

Train.TrainModel()
